In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from pathlib import Path
import peakutils
from peakutils.plot import plot as pplot
from matplotlib import pyplot
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path

import talib

In [5]:
def parse_trades(filepath):
    trades = pd.read_csv(filepath, header=0,
                         parse_dates=True, index_col=0)
    trades['open_date'] = pd.to_datetime(trades['open_date'])
    trades['close_date'] = pd.to_datetime(trades['close_date'])

    return trades

class CSVDataHandler:
    def __init__(self, csv_dir, symbol_list, timeframe):
        self.csv_dir = csv_dir
        self.symbol_list = symbol_list
        self.timeframe = timeframe
        
        self.symbol_data = dict()
        self.latest_symbol_data = dict()
        
        self.continue_backtest = True
        
        self._import_symbol_data()
        
    def _import_symbol_data(self):
        csv_files_path = f'{Path().absolute()}/{self.csv_dir}'
        columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
        combined_symbol_index = None
        
        for symbol in symbol_list:
            symbol_csv_path = f'{csv_files_path}/{symbol}_{self.timeframe}.csv'
            self.symbol_data[symbol] = pd.read_csv(symbol_csv_path, header=None, index_col=0, names=columns)
            
            self.symbol_data[symbol].index = pd.to_datetime(self.symbol_data[symbol].index, unit='ms')
            self.symbol_data[symbol] = self.symbol_data[symbol].drop_duplicates()
            self.symbol_data[symbol] = self.symbol_data[symbol].sort_index()
            
            if combined_symbol_index is None:
                combined_symbol_index = self.symbol_data[symbol].index
            else:
                combined_symbol_index = combined_symbol_index.union(self.symbol_data[symbol].index)
            
            self.latest_symbol_data[symbol] = list()
            
        for symbol in symbol_list:
            self.symbol_data[symbol] = self.symbol_data[symbol].reindex(index=combined_symbol_index, method='pad', fill_value=0).itertuples(name='OHLCV')
            
    def _get_new_bar(self, symbol):
        for bar in self.symbol_data[symbol]:
            yield bar
            
    def update_bars(self):
        for symbol in self.symbol_list:
            try:
                bar = next(self._get_new_bar(symbol))
            except StopIteration:
                self.continue_backtest = False
            else:
                if bar is not None:
                    self.latest_symbol_data[symbol].append(bar)
                    
    def get_latest_bar(self, symbol):
        try:
            return self.latest_symbol_data[symbol][-1]
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
            
    def get_latest_bars(self, symbol, N=1):
        try:
            return np.array(self.latest_symbol_data[symbol][-N:])
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
            
    def get_latest_bar_datetime(self, symbol):
        try:
            return self.latest_symbol_data[symbol][-1].Index
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
            
    def get_latest_bar_value(self, symbol, value_type):
        try:
            return getattr(self.latest_symbol_data[symbol][-1], value_type)
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
            
    def get_latest_bars_values(self, symbol, value_type, N=1):
        try:
            bars = self.latest_symbol_data[symbol][-N:]
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
        else:
            return np.array([getattr(bar, value_type) for bar in bars])

In [6]:
symbol_list = ['BTC-USDT']
data_handler = CSVDataHandler('../exchange_data', symbol_list, timeframe='1h')

In [7]:
for _ in range(250):
    data_handler.update_bars()

In [9]:
data_handler.get_latest_bars('BTC-USDT', N=100)

array([[Timestamp('2017-08-23 10:00:00'), 4208.11, 4228.02, 4188.63,
        4228.02, 24.062874],
       [Timestamp('2017-08-23 11:00:00'), 4203.12, 4237.99, 4174.09,
        4237.99, 33.082033],
       [Timestamp('2017-08-23 12:00:00'), 4205.98, 4265.8, 4174.0,
        4215.68, 62.638690000000004],
       [Timestamp('2017-08-23 13:00:00'), 4215.68, 4230.92, 4155.0,
        4211.0, 81.248254],
       [Timestamp('2017-08-23 14:00:00'), 4166.89, 4212.29, 4154.51,
        4174.0, 58.313942000000004],
       [Timestamp('2017-08-23 15:00:00'), 4174.0, 4226.95, 4150.65,
        4197.05, 35.159683],
       [Timestamp('2017-08-23 16:00:00'), 4226.94, 4259.58, 4197.05,
        4210.03, 71.149755],
       [Timestamp('2017-08-23 17:00:00'), 4220.56, 4231.0, 4166.27,
        4166.27, 39.170494],
       [Timestamp('2017-08-23 18:00:00'), 4166.27, 4191.71, 4135.38,
        4143.37, 31.864933],
       [Timestamp('2017-08-23 19:00:00'), 4143.37, 4168.9, 4103.51,
        4117.07, 32.452403000000004],
 

In [10]:
from market_profile import MarketProfile

In [11]:
data_handler.get_latest_bars_values('BTC-USDT', N=100)

TypeError: get_latest_bars_values() missing 1 required positional argument: 'value_type'